# 9.6 Homework: SQL and Databases
- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Question 1
Write the following queries in SQL, using the university schema.

1. Find the ID and name of each student who has taken at least one Comp. Sci. course; make sure there are no duplicate names in the result.
2. Find the ID and name of each student who has not taken any course offered before 2017.
3. For each department, find the maximum salary of instructors in that department. You may assume that every department has at least one instructor.
4. Find the lowest, across all departments, of the per-department maximum salary computed by the preceding query.

<img src="University Schema.png" width="1000">